In [105]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [106]:
from IPython.display import display, Markdown, Latex

In [107]:
from pathlib import Path
home = str(Path.home())

In [108]:
import sys
sys.path.insert(0,f'{home}/knowledgelib/')

In [109]:
import pyknowledge

In [110]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [111]:
import scipy.io
import pandas as pd
import numpy as np

mat = scipy.io.loadmat("/disk/metabric/BRCA1View20000.mat")

In [112]:
gene_labels = [g[0] for g in mat['gene'][0]]
df = pd.DataFrame(mat['data'].transpose(), columns=gene_labels)
# df['id'] = mat['id'][0]
df['target'] = mat['targets']
df['Subtype'] = df.target.map({1:'Basal',2:'HER2+',3:'LumA',4:'LumB',5:'Normal Like',6:'Normal'})
df.head()

,SCGB2A2,MUCL1,SCGB1D2,PIP,LOC648852,DNAJA2,TFF3,S100P,CPB1,CEACAM6,...,LOC645307,IL1RAP,LOC647149,LOC642453,LOC652100,LOC646050,LOC644912,LOC652294,target,Subtype
0,6.365374,5.610561,5.134799,5.735802,5.855315,5.443985,8.005882,5.615179,6.682096,5.756946,...,5.420998,5.396492,5.358751,5.424770,5.411055,5.242460,5.197767,5.643537,6,Normal
1,14.330442,7.979139,10.244240,7.138279,9.890661,11.195867,13.511164,10.548596,5.491578,6.805606,...,5.167652,5.273045,5.287980,5.398061,5.226934,5.396744,5.352158,5.811813,4,LumB
2,14.388331,6.016715,12.164435,5.272691,6.468386,5.265518,13.386034,8.764722,7.331663,11.967719,...,5.327874,5.549589,5.333051,5.368544,5.296182,5.694022,5.459744,5.269015,4,LumB
3,13.111243,7.251862,8.389121,8.664654,10.378585,5.544646,11.159766,8.263224,5.497906,5.923506,...,5.009093,5.345468,5.176322,5.326991,5.480259,5.292691,5.242160,5.635111,5,Normal Like
4,12.243103,7.293489,10.919581,9.566137,8.503568,11.492583,11.531352,9.818938,9.443134,6.889670,...,5.487783,5.479967,5.332074,5.273141,5.282872,5.222482,5.186136,5.616094,3,LumA


In [113]:
lum_genes = pd.read_csv('knowledgelib/metabric/Luminal_B_101/Luminal_B_101_Gene_List.csv')

genes_df = df.loc[:,df.columns.isin(lum_genes["Gene"])]

## Here is where we would select difference scalers

In [114]:
scaler = None

if scaler is None:
    scaler = StandardScaler()
    scaler.fit(genes_df)

genes_df_scaled = pd.DataFrame(scaler.transform(genes_df),index=genes_df.index,columns=genes_df.columns).fillna(0)

In [115]:
genes_df_scaled.head()

,CCND1,FAM83H,C17orf28,FAM83D,SRGAP1,UBE2C,BOP1,BRF2,KIFC2,TROAP,...,RAD54B,NFKBIL2,ZBTB10,PSMC3IP,LETM2,BRCA1,FAM82B,MYBL2,FRS2,CIT
0,-0.100659,-3.189314,-1.163106,-1.368472,0.197607,-2.160346,-1.268345,0.235924,-0.933807,-1.898709,...,-1.410917,-0.539807,-0.990863,-1.394209,-1.683707,-1.760110,0.504558,-0.916205,-1.437369,1.309344
1,-0.396379,1.923176,0.505856,-0.638042,1.477128,0.040077,2.057732,2.492817,2.267095,-0.076725,...,0.275690,0.731575,-0.143936,-0.736753,-0.378679,-0.281442,1.326211,1.086842,-0.746088,-0.877594
2,0.153928,2.076008,0.727836,-0.948505,0.592500,-0.527023,1.303371,0.617340,1.742926,-0.027667,...,-0.095986,-1.137496,-1.343864,-0.062959,0.889682,1.095956,2.489593,0.528456,0.021510,-0.872164
3,0.764704,0.830027,-0.425847,-0.142911,0.458682,0.967930,1.843483,-0.298788,1.120019,1.767628,...,-0.692801,1.483901,-1.242918,0.485040,-0.336100,1.149192,0.498185,-1.617987,0.403393,2.220044
4,1.547597,0.188069,1.024349,-0.520472,0.484273,0.318252,-0.363769,2.339653,0.595944,0.124801,...,-0.001107,-0.621344,-1.815289,1.112204,1.060896,1.191882,-1.432317,2.028444,0.153414,-0.619178


In [116]:
# Try Binning to Generate Unequal Training and Test Distributions

# Test Size of 10%
test_size = 200 #10%

In [117]:
# Check how many of genes are from different populations

from scipy import stats 
def similarity(df1, df2):
    #print(stats.ttest_ind(df1, df2))
    return len([i for i in stats.ttest_ind(df1, df2).pvalue if i < 0.05])

In [203]:
# Bin each Gene

binned = pd.qcut(genes_df_scaled.values.reshape(191970), 10, retbins=True)[1]
binned = [-100] + list(binned[1:-1]) + [100]
genes_df_scaled_binned = pd.DataFrame()
for i in genes_df_scaled.columns:
    genes_df_scaled_binned[i] = pd.cut(genes_df_scaled[i], bins=binned).astype(str)

In [236]:
import itertools 

d = {}

for a in list(set(genes_df_scaled_binned.values.reshape(191970))):
    selected = (genes_df_scaled_binned == a).sum(axis=1).sort_values(ascending=False)[0:test_size]
    if(selected.values[-1] == 0):
        print(selected)
    test_ids = selected.index
    
    # If number of genes from different populations is close to 90, 
    # we can ensure the test set is different enough from the training set 

    s = similarity(genes_df_scaled.iloc[test_ids], genes_df_scaled.iloc[~genes_df_scaled.index.isin(test_ids)])
    if (s > 80):
        train_ids = [i for i in genes_df_scaled.index if i not in test_ids]
        d[a] = {'train_ids': list(train_ids), 'test_ids': list(test_ids)}
        


In [237]:
import json
with open("shiftedsets.json", "w") as outfile: 
    json.dump(d, outfile)

{'(-1.113, -0.788]': {'train_ids': [0,
   1,
   2,
   3,
   4,
   5,
   6,
   7,
   9,
   11,
   13,
   14,
   15,
   16,
   17,
   18,
   19,
   20,
   22,
   24,
   26,
   28,
   29,
   30,
   31,
   32,
   33,
   34,
   35,
   36,
   38,
   39,
   40,
   41,
   43,
   44,
   45,
   46,
   47,
   48,
   49,
   50,
   51,
   52,
   53,
   55,
   56,
   57,
   58,
   59,
   60,
   61,
   62,
   63,
   64,
   65,
   66,
   67,
   68,
   69,
   70,
   71,
   72,
   73,
   74,
   75,
   76,
   77,
   78,
   79,
   80,
   81,
   82,
   83,
   85,
   86,
   88,
   89,
   90,
   91,
   92,
   93,
   94,
   95,
   97,
   98,
   99,
   100,
   101,
   102,
   103,
   105,
   106,
   107,
   108,
   109,
   110,
   111,
   112,
   113,
   114,
   115,
   116,
   117,
   118,
   119,
   121,
   122,
   124,
   125,
   126,
   127,
   128,
   129,
   130,
   131,
   132,
   133,
   134,
   135,
   137,
   138,
   139,
   140,
   141,
   142,
   144,
   145,
   146,
   147,
   148,
   149,
   150,